In [1]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from ces_fun_util import *
#from fun_utils_eq import *

In [2]:
## parameter values
alpha = 0.85           # labor share parameter in manufacturing
#alpha = 0.2
theta = 4             # scopevec for comparative advantage
sigma = 1      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1  # elasticity of demand for each individual manufactured good j at Foreign
epsilonD = alpha + (1 - alpha) * sigma  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar  #Foreign's elasticity of demand for embodied energy
#beta = 2.274853
#gamma= 0.784877595
beta=1.892412
gamma=0.807998928
epsilonS = 0.5  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5  #Foreign's energy supply elasticity: betastar/(1 - betastar)

In [3]:
## scenario switch
case=3 # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0 # 1 means logit estimations of supply elasticity; 0 means fixed elasticities at 0.5

In [4]:
ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)

In [5]:
class taxrow_eq:
    def __init__(self, pe, te, tb_mat, df, tax_scenario, varphi, ParaList):
        self.df = df
        self.tax_scenario = tax_scenario
        self.varphi = varphi
        self.ParaList = ParaList
        self.prop = 1
        self.te = te
        self.pe = pe
        self.tb_mat = tb_mat

    ## computes equilibrium price and taxes. Also computes other equilibrium values (ie consumption, production, value of exports/imports)
    ## and stores them in self. 
    def opt_tax(self):
        tax_scenario = self.tax_scenario
        varphi = self.varphi
        df = self.df
        te = varphi
        self.region = df['region_scenario']
        pes = np.append([self.pe], np.arange(0.1, 2, 0.2))
        tbs = np.append([self.tb_mat[0]], np.arange(0,1.5,0.2))
        props = np.append([self.tb_mat[1]], np.arange(0,1.1,0.5))
        self.conv = 1
        if (tax_scenario['tax_sce'] == 'global'):
            tbs = [0]
        
        if tax_scenario['tax_sce'] != 'purete' and tax_scenario['tax_sce'] != 'EP_hybrid':
            for prop in props:
                for tb in tbs:
                    for pe in pes:
                        res = minimize(self.minuswelfare, [pe, tb, prop], bounds=[(0.01, np.inf), (0, np.inf), (0, 1)],
                               method='nelder-mead', args=(te), tol=0.000001, options={'maxfev': 100000})
                        if res.fun <= 0.0001:
                            opt_val = res.x
                            break
                    else:
                        continue
                    break
                else: 
                    continue
                break
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            opt_val = res.x
            #print(res.fun, df['region_scenario'], tax_scenario['tax_sce'], varphi)
            
        pes = np.append([self.pe], np.arange(0.1, 2, 0.5))
        props = np.append([self.tb_mat[1]], np.arange(0,2,0.5))
        if tax_scenario['tax_sce'] == 'EP_hybrid':
            for pe in pes:
                for tb in tbs:
                    for prop in props:
                        res = minimize(self.minuswelfare, [pe, tb, prop], bounds=[(0.01, np.inf), (0, np.inf), (0, np.inf)],
                               method='nelder-mead', args=(te), tol=0.000001, options={'maxfev': 100000})
                        if res.fun <= 0.0001:
                            opt_val = res.x
                            break
                    else:
                        continue
                    break
                else: 
                    continue
                break
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            opt_val = res.x
            
            
        ## not implemented yet
        if tax_scenario['tax_sce'] == 'Baseline':
            self.tb = 0
            self.prop = 1
            self.te = 0
        
        elif tax_scenario['tax_sce'] == 'global':
            self.tb = 0
            self.te = varphi

        elif tax_scenario['tax_sce'] == 'PC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]
            self.te = self.tb

        elif tax_scenario['tax_sce'] == 'EPC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]
            self.te = varphi

        elif tax_scenario['tax_sce'] == 'EP_hybrid':
            self.tb = opt_val[1]
            self.te = opt_val[2]
            self.prop = self.te #- self.tb

        elif tax_scenario['tax_sce'] == 'Unilateral' or tax_scenario['tax_sce'] == 'puretc' or tax_scenario[
            'tax_sce'] == 'puretp' or tax_scenario['tax_sce'] == 'EC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]

            if tax_scenario['tax_sce'] == 'puretc' or tax_scenario['tax_sce'] == 'puretp':
                self.te = self.tb

            elif tax_scenario['tax_sce'] == 'Unilateral' or tax_scenario['tax_sce'] == 'EC_hybrid':
                self.te = self.varphi

        elif tax_scenario['tax_sce'] == 'purete':
            res = minimize(self.minuswelfare, [self.pe, self.te], bounds=[(0.001, np.inf), (0, np.inf)], method='nelder-mead', args=(0),
                           tol=0.000001, options={'maxfev': 100000})
            
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            #print(res.fun)
            self.te = res.x[1]
            tb_mat = [0, 1]
            self.tb = tb_mat[0]
            self.prop = tb_mat[1]
        #print(tax_scenario['tax_sce'], res)

    def minuswelfare(self, p, te):
        pe = p[0]
        tb_mat = p[1:]
        varphi = self.varphi
        tax_scenario = self.tax_scenario
        ParaList = self.ParaList
        df = self.df

        if tax_scenario['tax_sce'] == 'purete':
            te = p[1]
            tb_mat = [0, 1]

        obj = self.comp_obj(pe, te, tb_mat, varphi, tax_scenario, ParaList, df)

        return obj

    ## compute the objective value, currently the objective is to minimize negative welfare
    ## also saves optimal results in self.
    def comp_obj(self, pe, te, tb_mat, varphi, tax_scenario, Paralist, df):

        alpha, theta, sigma, sigmastar, epsilonD, epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList
        ## compute extraction tax, and jbar's
        te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = computejbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df)

        jxbar_prime = jxbar_hat * df['jxbar']
        jmbar_prime = jmbar_hat * df['jmbar']
        j0_prime = j0_hat * df['jxbar']

        jvals = (j0_hat, j0_prime, jxbar_hat, jxbar_prime, jmbar_hat, jmbar_prime)

        # if te is too large, Home stop producing
        petbte = pe + tb_mat[0] - te
        z = pe + tb_mat[0] >= te
        petbte = petbte * z

        Qe_prime, Qestar_prime = compute_qe(tax_scenario, petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)

        Cey_prime, Cex1_prime, Cex2_prime, Cex_prime, Cem_prime, CeFF_prime, Cex_hat, Cex1_hat, Cem_hat = comp_ce(
            pe, tb_mat, jvals, ParaList, df, tax_scenario)

        Ge_prime = Cey_prime + Cex_prime
        Gestar_prime = CeFF_prime + Cem_prime
        Ce_prime = Cey_prime + Cem_prime
        Cestar_prime = CeFF_prime + Cex_prime
        Qeworld_prime = Qe_prime + Qestar_prime

        consvals = (Cey_prime, Cex1_prime, Cex2_prime, Cex_prime, 
                    Cem_prime, CeFF_prime, Cex_hat, Cex1_hat, Cem_hat,
                    Ce_prime, Cestar_prime)

        Vgy, Vgystar, Vgx1_prime, Vgx2_prime, Vgx_prime, Vgm_prime, Vgx, Vgm = comp_vg(pe, tb_mat, jvals, consvals,
                                                                                           df, tax_scenario, ParaList)

        pai_g = Vgx - (pe + tb_mat[0]) * df['CeFH'] / (1 - alpha)
        subsidy_ratio = 1- ((1-jxbar_prime) * j0_prime / ((1-j0_prime) * jxbar_prime))**(1/theta)

        ## compute Ve values
        Ve_prime, Vestar_prime = comp_ve(pe, tb_mat, consvals, tax_scenario)

        ## compute more Vg values
        vgvals = Vgx_prime, Vgm_prime
        Vg, Vg_prime, Vgstar, Vgstar_prime = comp_vgfin(pe, tb_mat, consvals, vgvals, jvals, ParaList, df, tax_scenario)

        Lg, Lg_prime, Lgstar, Lgstar_prime = comp_lg(pe, tb_mat, Ge_prime, Gestar_prime, consvals, ParaList, df, tax_scenario)

        leakage1, leakage2, leakage3 = comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df)

        lgvals = Lg, Lgstar, Lg_prime, Lgstar_prime
        vgvals = Vg, Vgstar, Vg_prime, Vgstar_prime
        delta_Le, delta_Lestar, delta_U = comp_delta(lgvals, vgvals, Qeworld_prime, df, jvals, pe, petbte, tb_mat, tax_scenario, varphi, ParaList)

        chg_extraction, chg_production, chg_consumption, chg_Qeworld = comp_chg(df, Qestar_prime, Gestar_prime,
                                                                                Cestar_prime, Qeworld_prime)

        welfare = delta_U / Vg * 100
        welfare_noexternality = (delta_U + varphi * (Qeworld_prime - df['Qeworld'])) / Vg * 100
        
        self.results = assign_val(Ge_prime, Gestar_prime, Lg_prime, Lgstar_prime, Qe_prime, Qestar_prime, Qeworld_prime,
                                  Ve_prime, Vestar_prime, Vgx1_prime, Vgx2_prime, Vgx_prime, Vgm_prime, Vg_prime,
                                  Vgstar_prime, chg_Qeworld, chg_consumption, chg_extraction, chg_production, delta_Le, 
                                  delta_Lestar, leakage1, leakage2, leakage3,
                                  pai_g, pe, subsidy_ratio, varphi, welfare, welfare_noexternality, jvals, consvals)

        diff, diff1, diff2 = comp_diff(consvals, jvals, Ge_prime, Gestar_prime, Qe_prime, Qestar_prime, Qeworld_prime,
                                       Vgx2_prime, pe, tax_scenario, tb_mat, te, varphi, ParaList, df)

        obj_val = abs(diff1) + abs(diff) + abs(diff2)
        return obj_val

    ## retrive the pandas series object containing equilibrium values (prices, taxes, consumption etc)
    def retrive(self):
        ret = self.results
        ret['tb'] = self.tb
        ret['prop'] = self.prop
        ret['te'] = self.te
        ret['region_scenario'] = self.region
        ret['conv'] = self.conv
        
        return ret

In [6]:
## import BAU values (seven regional scenarios in the order of US, EU, OECD, World, China, OECD plus China)
if case==2:
    df = pd.read_csv("../../output/BaselineCarbon_2015_noTradeinGoods.csv",index_col=['region_scenario','regionbase'],header='infer')
elif case==3:
    df = pd.read_csv("../../output/BaselineCarbon_2015.csv", index_col=['regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF'])
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF'])

## choose which regional scenario to run (runs all if not executed)
#df=df.drop([2,3,4,5,6,7])  
#df = df[df['region_scenario'] == 1]
df = df[df['region_scenario'] != 4]
df

,region_scenario,Qe,Qestar,Qeworld,CeHH,CeHF,CeFH,CeFF,Ce,Cestar,Ge,Gestar,Ceworld,Geworld,jxbar,jmbar
regionbase,,,,,,,,,,,,,,,,
US as Home,1,4.48004,27.79594,32.27598,4.59840,1.19611,0.42160,26.05987,5.79451,26.48147,5.02000,27.25598,32.27598,32.27598,0.015921,0.793579
EU28 as Home,2,0.93578,31.34020,32.27598,2.95058,1.01356,0.50767,27.80417,3.96414,28.31184,3.45825,28.81773,32.27598,32.27598,0.017931,0.744318
OECD37 as Home,3,8.62549,23.65050,32.27599,11.29367,2.48754,0.91057,17.58421,13.78121,18.49478,12.20424,20.07175,32.27599,32.27599,0.049234,0.819498
China as Home,5,7.52274,24.75324,32.27598,7.34547,0.63247,1.93538,22.36266,7.97794,24.29804,9.28085,22.99513,32.27598,32.27598,0.079652,0.920723
OECD and China as Home,6,16.14824,16.12775,32.27599,20.10318,1.65596,1.38191,9.13494,21.75914,10.51685,21.48509,10.79090,32.27599,32.27599,0.131400,0.923896
US and EU as Home,7,5.41582,26.86015,32.27597,7.72887,2.02977,0.74938,21.76795,9.75864,22.51733,8.47825,23.79772,32.27597,32.27597,0.033280,0.792003


In [7]:
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','global','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[0,1,1,1,1,1,1,1,1]},index=[1,2, 3, 4, 5, 6, 7, 8,9])
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[0,1,1,1,1,1,1,1]},index=[1,2, 3, 4, 5, 6, 7, 8])
#tax_scenario= pd.DataFrame({'tax_sce': ['PC_hybrid','EPC_hybrid'], 'Base':[1,1]},index=[1,2])
#tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral'], 'Base':[0]},index=[1])
#df = pd.DataFrame({'Qe': 4.4800448, 'Qestar': 27.795946, 'Qeworld': 32.27599, 'CeHH': 4.598403, 'CeHF': 1.196111,'CeFH': 0.42160103,'CeFF': 26.059874,'Ce': 5.7945137,'Cestar': 26.481474,'Ge': 5.0200038,'Gestar': 27.255985,'Ceworld': 32.27599,'Geworld': 32.27599,'jxbar': 0.015920602214279298,'jmbar':  0.7935787194577492}, index = [1])

In [8]:
def temp(tax_scenario, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit):
    ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)
    tb_mat = [0, 1];  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)
    # use for quick test: varphilist = [2] or varphilist = np.arange (1.7,2.5,0.1)
    #varphilist = np.arange(0,3,0.1)
    varphilist = np.arange (0,10,0.1)
    #varphilist = [2.9]
    output=[]
    prevtb = [1,1,1,1]
    for varphi in varphilist:    
        tax_df=df.apply(mintemp, axis=1, raw=False, args=(prevtb, varphi, tax_scenario, ParaList))
        prevtb = tax_df[['region_scenario','tb', 'pe', 'te', 'prop']]
        output.append(tax_df)
        print(varphi)
    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid':
        output.te[output.Qe_prime==0]=output.pe+output.tb
    print(tax_scenario['tax_sce'])
    return output

In [9]:
def mintemp(df, prevtb, varphi, tax_scenario, ParaList):
    pe = 0.5
    te = 0.5
    tb_mat = [0.5,0.5]
    #print(prevtb['region_scenario'])
    
    if (type(prevtb) != list):
        curr_region = prevtb[prevtb['region_scenario'] == df['region_scenario']]
        vals = curr_region.values
        #print(vals)
        tb_mat = [vals[0][1], vals[0][4]]
        pe = vals[0][2]
        te = vals[0][3]
    tax_temp = taxrow_eq(pe, te, tb_mat, df, tax_scenario, varphi, ParaList)
    tax_temp.opt_tax()
    ret = tax_temp.retrive()
    #print(ret)
    return ret

In [10]:
temp_df = tax_scenario.apply(temp, axis=1, args=(alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))
#temp_df.head()

0.0
0.1
0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.8
0.9
1.0
1.1
1.2000000000000002
1.3
1.4000000000000001
1.5
1.6
1.7000000000000002
1.8
1.9000000000000001
2.0
2.1
2.2
2.3000000000000003
2.4000000000000004
2.5
2.6
2.7
2.8000000000000003
2.9000000000000004
3.0
3.1
3.2
3.3000000000000003
3.4000000000000004
3.5
3.6
3.7
3.8000000000000003
3.9000000000000004
4.0
4.1000000000000005
4.2
4.3
4.4
4.5
4.6000000000000005
4.7
4.800000000000001
4.9
5.0
5.1000000000000005
5.2
5.300000000000001
5.4
5.5
5.6000000000000005
5.7
5.800000000000001
5.9
6.0
6.1000000000000005
6.2
6.300000000000001
6.4
6.5
6.6000000000000005
6.7
6.800000000000001
6.9
7.0
7.1000000000000005
7.2
7.300000000000001
7.4
7.5
7.6000000000000005
7.7
7.800000000000001
7.9
8.0
8.1
8.200000000000001
8.3
8.4
8.5
8.6
8.700000000000001
8.8
8.9
9.0
9.1
9.200000000000001
9.3
9.4
9.5
9.600000000000001
9.700000000000001
9.8
9.9
Unilateral
0.0
0.1
0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.70000000000000

C:\Users\Alex\Desktop\Tobin\sam\Optimal-Unilateral-Carbon-Policy\Data\codes\ces\ces_fun_util.py:457: RuntimeWarning: invalid value encountered in double_scalars
  leakage1 = -(Qestar_prime - df['Qestar']) / (Qeworld_prime - df['Qeworld'])
C:\Users\Alex\Desktop\Tobin\sam\Optimal-Unilateral-Carbon-Policy\Data\codes\ces\ces_fun_util.py:458: RuntimeWarning: divide by zero encountered in double_scalars
  leakage2 = -(Gestar_prime - df['Gestar']) / (Qeworld_prime - df['Qeworld'])
C:\Users\Alex\Desktop\Tobin\sam\Optimal-Unilateral-Carbon-Policy\Data\codes\ces\ces_fun_util.py:459: RuntimeWarning: divide by zero encountered in double_scalars
  leakage3 = -(Cestar_prime - df['Cestar']) / (Qeworld_prime - df['Qeworld'])


0.1
0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.8
0.9
1.0
1.1
1.2000000000000002
1.3
1.4000000000000001
1.5
1.6
1.7000000000000002
1.8
1.9000000000000001
2.0
2.1
2.2
2.3000000000000003
2.4000000000000004
2.5
2.6
2.7
2.8000000000000003
2.9000000000000004
3.0
3.1
3.2
3.3000000000000003
3.4000000000000004
3.5
3.6
3.7
3.8000000000000003
3.9000000000000004
4.0
4.1000000000000005
4.2
4.3
4.4
4.5
4.6000000000000005
4.7
4.800000000000001
4.9
5.0
5.1000000000000005
5.2
5.300000000000001
5.4
5.5
5.6000000000000005
5.7
5.800000000000001
5.9
6.0
6.1000000000000005
6.2
6.300000000000001
6.4
6.5
6.6000000000000005
6.7
6.800000000000001
6.9
7.0
7.1000000000000005
7.2
7.300000000000001
7.4
7.5
7.6000000000000005
7.7
7.800000000000001
7.9
8.0
8.1
8.200000000000001
8.3
8.4
8.5
8.6
8.700000000000001
8.8
8.9
9.0
9.1
9.200000000000001
9.3
9.4
9.5
9.600000000000001
9.700000000000001
9.8
9.9
puretc
0.0
0.1
0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.8
0

C:\Users\Alex\Desktop\Tobin\sam\Optimal-Unilateral-Carbon-Policy\Data\codes\ces\ces_fun_util.py:457: RuntimeWarning: divide by zero encountered in double_scalars
  leakage1 = -(Qestar_prime - df['Qestar']) / (Qeworld_prime - df['Qeworld'])


0.1
0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.8
0.9
1.0
1.1
1.2000000000000002
1.3
1.4000000000000001
1.5
1.6
1.7000000000000002
1.8
1.9000000000000001
2.0
2.1
2.2
2.3000000000000003
2.4000000000000004
2.5
2.6
2.7
2.8000000000000003
2.9000000000000004
3.0
3.1
3.2
3.3000000000000003
3.4000000000000004
3.5
3.6
3.7
3.8000000000000003
3.9000000000000004
4.0
4.1000000000000005
4.2
4.3
4.4
4.5
4.6000000000000005
4.7
4.800000000000001
4.9
5.0
5.1000000000000005
5.2
5.300000000000001
5.4
5.5
5.6000000000000005
5.7
5.800000000000001
5.9
6.0
6.1000000000000005
6.2
6.300000000000001
6.4
6.5
6.6000000000000005
6.7
6.800000000000001
6.9
7.0
7.1000000000000005
7.2
7.300000000000001
7.4
7.5
7.6000000000000005
7.7
7.800000000000001
7.9
8.0
8.1
8.200000000000001
8.3
8.4
8.5
8.6
8.700000000000001
8.8
8.9
9.0
9.1
9.200000000000001
9.3
9.4
9.5
9.600000000000001
9.700000000000001
9.8
9.9
PC_hybrid
0.0
0.1
0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.

In [11]:
output_list=[]
for i in range(1,len(tax_scenario)+1):
    output_list.append(temp_df.loc[i])
Outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
Outcomes.reset_index(level=0, inplace=True)
Outcomes

,tax_sce,regionbase,varphi,pe,tb,prop,te,jxbar_prime,jmbar_prime,j0_prime,...,chg_extraction,chg_production,chg_consumption,chg_Qeworld,pai_g,subsidy_ratio,welfare,welfare_noexternality,region_scenario,conv
0,Unilateral,US as Home,0.0,1.000000,0.000000,0.770755,0.0,0.015921,0.793579,0.015921,...,-2.751366e-07,3.867016e-07,3.757130e-07,-3.194820e-07,5.564246e-08,0.000000,0.000002,0.000002,1.0,1.0
6,Unilateral,US as Home,0.1,0.998648,0.042625,0.000000,0.1,0.015925,0.793579,0.015530,...,-1.880037e-02,-9.025102e-03,1.326422e-02,-1.523408e-01,-1.160040e-01,0.006359,0.019772,-0.019664,1.0,1.0
12,Unilateral,US as Home,0.2,0.997713,0.085194,0.000017,0.2,0.015939,0.793579,0.015160,...,-3.180472e-02,-2.410336e-02,1.948608e-02,-3.022594e-01,-2.330233e-01,0.012638,0.078665,-0.077824,1.0,1.0
18,Unilateral,US as Home,0.3,0.997174,0.127742,0.000011,0.3,0.015961,0.793579,0.014809,...,-3.930055e-02,-4.503584e-02,1.894037e-02,-4.503485e-01,-3.510981e-01,0.018839,0.176105,-0.173634,1.0,1.0
24,Unilateral,US as Home,0.4,0.997015,0.170303,0.000008,0.4,0.015990,0.793579,0.014475,...,-4.151012e-02,-7.168104e-02,1.183771e-02,-5.971816e-01,-4.702771e-01,0.024961,0.311711,-0.306648,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,EPC_hybrid,US and EU as Home,9.5,0.948055,4.195562,0.788111,9.5,0.021912,0.792003,0.009552,...,-7.069252e-01,-3.017269e-01,6.826033e-01,-6.122745e+00,-2.070096e+01,0.190002,71.360905,-18.046142,7.0,1.0
581,EPC_hybrid,US and EU as Home,9.6,0.947481,4.237976,0.789457,9.6,0.021882,0.792003,0.009477,...,-7.148397e-01,-2.947989e-01,6.924493e-01,-6.130660e+00,-2.090999e+01,0.191322,72.302636,-18.162326,7.0,1.0
587,EPC_hybrid,US and EU as Home,9.7,0.946916,4.280379,0.790787,9.7,0.021852,0.792003,0.009403,...,-7.226415e-01,-2.879641e-01,7.021638e-01,-6.138462e+00,-2.111900e+01,0.192635,73.245587,-18.278042,7.0,1.0
593,EPC_hybrid,US and EU as Home,9.8,0.946359,4.322770,0.792100,9.8,0.021823,0.792003,0.009330,...,-7.303319e-01,-2.812223e-01,7.117479e-01,-6.146152e+00,-2.132800e+01,0.193939,74.189717,-18.393300,7.0,1.0


In [12]:
Outcomes[['pe','tb', 'Cey_prime', 'Cex_prime', 'Cem_prime', 'Ceystar_prime', 'Vgx_prime', 'Vgm_prime']]

,pe,tb,Cey_prime,Cex_prime,Cem_prime,Ceystar_prime,Vgx_prime,Vgm_prime
0,1.000000,0.000000,4.598400,0.421600,1.196110,26.059870,2.810667,7.974067
6,0.998648,0.042625,4.466978,0.409705,1.161925,26.085030,2.811499,7.974067
12,0.997713,0.085194,4.343070,0.398774,1.129695,26.102182,2.813905,7.974067
18,0.997174,0.127742,4.225981,0.388705,1.099239,26.111706,2.817762,7.974067
24,0.997015,0.170303,4.115094,0.379404,1.070395,26.113904,2.822960,7.974067
...,...,...,...,...,...,...,...,...
575,0.948055,4.195562,2.339016,0.318217,0.614277,22.881716,3.289357,13.531800
581,0.947481,4.237976,2.324949,0.317441,0.610582,22.892339,3.284841,13.531800
587,0.946916,4.280379,2.311076,0.316677,0.606939,22.902817,3.280394,13.531800
593,0.946359,4.322770,2.297395,0.315926,0.603346,22.913152,3.276016,13.531800


In [13]:
#outcome_t = Outcomes[Outcomes['regionbase'] == 'US as Home']
Outcomes[['pe','tb', 'Qe_prime', 'Qestar_prime', 'Ce_prime', 'Cestar_prime','Lg_prime','Lgstar_prime', 'Vg_prime', 'Vgstar_prime', 'Vgx2_prime', 'delta_Le','delta_Lestar','welfare','welfare_noexternality']]

,pe,tb,Qe_prime,Qestar_prime,Ce_prime,Cestar_prime,Lg_prime,Lgstar_prime,Vg_prime,Vgstar_prime,Vgx2_prime,delta_Le,delta_Lestar,welfare,welfare_noexternality
0,1.000000,0.000000,4.480040,27.795940,5.794510,26.481470,28.446667,154.450553,38.630067,176.543133,0.000000,-4.434543e-08,-2.751366e-07,0.000002,0.000002
6,0.998648,0.042625,4.346500,27.777140,5.628903,26.494734,28.389766,154.446065,38.630067,176.543133,0.069737,-1.295995e-01,-1.878766e-02,0.019772,-0.019664
12,0.997713,0.085194,4.209585,27.764135,5.472765,26.500956,28.335814,154.437458,38.630067,176.543133,0.137462,-2.544562e-01,-3.176834e-02,0.078665,-0.077824
18,0.997174,0.127742,4.068992,27.756639,5.325220,26.500410,28.284596,154.424968,38.630067,176.543133,0.203348,-3.744874e-01,-3.924501e-02,0.176105,-0.173634
24,0.997015,0.170303,3.924368,27.754430,5.185490,26.493308,28.235914,154.408782,38.630067,176.543133,0.267550,-4.895996e-01,-4.144816e-02,0.311711,-0.306648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,0.948055,4.195562,0.000000,26.153225,2.953292,23.199933,42.199575,135.505243,65.057600,150.115533,1.855513,-1.805273e+00,-6.884830e-01,71.360905,-18.046142
581,0.947481,4.237976,0.000000,26.145310,2.935531,23.209779,42.170705,135.506277,65.057600,150.115533,1.862219,-1.805273e+00,-6.959841e-01,72.302636,-18.162326
587,0.946916,4.280379,0.000000,26.137508,2.918016,23.219494,42.142062,135.507243,65.057600,150.115533,1.868835,-1.805273e+00,-7.033740e-01,73.245587,-18.278042
593,0.946359,4.322770,0.000000,26.129818,2.900741,23.229078,42.113643,135.508143,65.057600,150.115533,1.875365,-1.805273e+00,-7.106540e-01,74.189717,-18.393300


In [14]:
Outcomes.to_csv('../../output_sig0/ces-05.csv'.format(theta, case), header=True)